In [1]:
import numpy

### Aşağıdaki metot ile Numpy karmaşık sayıları oluşturulabilir. 

In [2]:
# This value changes the floating point precise bits in Numpy complex values.
# Use only 64, 128 or 256 bit for options.
using_complex_bits = 128

def c(*args, **kwargs) -> numpy.complexfloating:
    """
    Creates Numpy complex values.
        
    Example:
    ==============================================
        c(1 + 1j)
        c(10 + x*1j)
    """
    
    if using_complex_bits == 64:
        v = numpy.complex64(*args, **kwargs)
    elif using_complex_bits == 128:
        v = numpy.complex128(*args, **kwargs)
    elif using_complex_bits == 256:
        v = numpy.complex256(*args, **kwargs)
    else:
        raise ValueError(f"{using_complex_bits} is not supported in Numpy for complex bits!")
    return v

### Karmaşık sayıların modunu almaktadır. Bu kodda mod integer olmalıdır. Aynı zamanda bir karmaşık sayısının modunun pozitif denkliklerini bulan kod bulunmaktadır.

In [3]:
def mod_complex(z: numpy.complexfloating, m: int) -> numpy.complexfloating:
    """
    Calculates the a+bi(modk).
    z: Complex value
    m: Module
    """
        
    x = z/m
    x = (numpy.round(x.real) + (numpy.round(x.imag)*1j))*m
    return z-x

def mod_complex_positive(z: numpy.complexfloating, m: int) -> numpy.complexfloating:
    """
    Translates negative complex value to modulo of positive ones. 
    """
    
    real, imag = z.real, z.imag
    if z.imag < 0:
        imag = (z.imag + m)
    if z.real < 0:
        real = z.real + m
    return c(real + imag*1j)
    
def mod_complex_apply(z: numpy.ndarray, m: int) -> numpy.ndarray:
    """
    Applies complex mod to all matrix elements.
    """
    
    return numpy.vectorize(mod_complex)(z, m)

def mod_complex_positive_apply(z: numpy.ndarray, m: int) -> numpy.ndarray:
    """
    Translates positive modulo of all item on complex matrix.
    """
    
    return numpy.vectorize(mod_complex_positive)(z, m)

### Alfabe karakterleri için dönüşüm tablosu bulunmaktadır.

In [4]:
# This conversion table used for encryption of chars.
__conversion_table = {
    "A": 1, "B": 2, "C": 3, "D": 4, 
    "E": 5, "F": 6, "G": 7, "H": 8, 
    "I": 9, "J": 10, "K": 11, "L": 12, 
    "M": 13, "N": 14, "O": 15, "P": 16, 
    "Q": 17, "R": 18, "S": 19, "T": 20,
    "U": 21, "V": 22, "W": 23, "X": 24, 
    "Y": 25, "Z": 0
}

def table_length() -> int:
    return __conversion_table.__len__()

def get_key(char: str) -> int:
    """
    Gets the key of the char.
    """
    
    try:
        v = __conversion_table[char]
    except KeyError:
        raise KeyError(f"Char {char} is not in conversion table!")
    return v

def get_value(val: int) -> str:
    """
    Gets the key of int from dict.
    """
    
    try:
        c = list(__conversion_table.keys())[list(__conversion_table.values()).index(val)]
    except KeyError or ValueError:
        raise AttributeError(f"Value {val} is not in the conversion table.")
    return c

### Aşağıdaki metot bir metinin karmaşık sayı matrisine halinde kodlanmasını yapmaktadır.

In [5]:
def create_enc_complex_matrix(message: str) -> numpy.ndarray:
    """
    Creates the encrypted complex matrix P from message.
    """
    
    # Format the message.
    message = message.upper()
    message = message.replace(" ", "")
    complex_list = list()
    
    # Creates list of complex values.
    # If message length is single add an "0j" to end.
    for m in range(0, message.__len__(), 2):
        _r = get_key(message[m])
        try:
            _c = get_key(message[m+1])
        except KeyError:
            _c = 0
            print("Message length is not single. Adding a 0j in the end.")
        _cpx = c(_r + _c*1j)
        complex_list.append(_cpx)
        
    P = numpy.array(complex_list).reshape(2, int(complex_list.__len__()/2))
    return P

### Aşağıdaki metot yardımıyla yukarıdaki metot ile matris halinde kodlanmış metini çözer.

In [6]:
def decipher_enc_complex_matrix(matrix: numpy.ndarray) -> str:
    """
    Deciphers encrypted complex matrtix that created function from "create_enc_complex_matrix"
    """
    
    s = str()
    for i in range(matrix.__len__()):
        for j in range(matrix[i].__len__()):
            _r = get_value(matrix[i][j].real)
            _c = get_value(matrix[i][j].imag)
            s += _r + _c
    return s

### Bir 2x2 karmaşık sayı matrisinin determinantının ve coefficent matrisinin modunu bulur.  

In [7]:
def inverse_of_determinant_mod(complex_matrix: numpy.ndarray, modulo: int) -> numpy.complexfloating:
    """
    This method calculates inverse of [det(K)] in mod.
    
    Returns:
        Inverse of det(K), coefficent matrix.
        
    Notes:
        This function works if matrix 2x2.
    """
    
    z = numpy.linalg.det(complex_matrix)
    z = mod_complex(z, modulo)
    a, b = z.real, z.imag
    e = (numpy.power(a, 2) + numpy.power(b, 2))%modulo
    
    # Cheap way to calculate inverse of number in a spesific modulo.
    t = 0
    for i in range(1, 100):        
        _e = numpy.power(e, i)%modulo
        if _e == 1.0:
            break
        t = _e
    
    if t == 0:
        raise ValueError("Frequency of number in module calculation failed.")
        
    _c = (a*t)%modulo
    _d = (-b*t)%modulo
    coefficent = numpy.array([
        [complex_matrix[1][1], -complex_matrix[0][1]], [-complex_matrix[1][0], complex_matrix[0][0]]]
    )
    return c(_c + _d*1j), coefficent

### Mesaj bu metot ile kodlanmaktadır. Mesaj gönderen taraf bu metot ile mesajını şifreler ve şifrelenmiş mesaj geri döner.

In [8]:
def encryipt_message(message: str, modulo: int, shared_key: numpy.ndarray) -> str:
    """
    Encryipts the message.
    
    Args:
        message: Message to encryipt.
        modulo: Modulo.(Length of table)
        shared_key: Key K.
    """
    
    # Create a complex matrix from message. 
    P = create_enc_complex_matrix(message)
    print(f"Encrypted Complex Matrix: \n {P}")
    
    # Create C=KP(mod(modulo)). 
    # And Make sure everyone has positive modulo.
    C = mod_complex_apply(shared_key.dot(P), modulo)
    C = mod_complex_positive_apply(C, modulo)
    print(f"Created C matrix as: \n{C}")
    
    enc_message = decipher_enc_complex_matrix(C)
    print(f"Encrypted message as: {enc_message}")
    return enc_message

### Alıcı aşağıdaki metot ile şifrelenmiş mesajı çözer.

In [9]:
def decrypt_message(shared_key: numpy.ndarray, modulo: int, enc_message: str):
    """
    Decrypts the crypted message.
    Args:
        shared_key: K
        modulo: mod
        enc_message: Encrypted message.
    """
    
    idm, coefficent = inverse_of_determinant_mod(shared_key, modulo)
    inv_K = mod_complex_positive_apply(mod_complex_apply(idm*coefficent, modulo), modulo)
    print(f"Inverse of K(mod): \n{inv_K}")
    C = create_enc_complex_matrix(enc_message)
    P = inv_K.dot(C)
    P = mod_complex_apply(P, modulo)
    P = mod_complex_positive_apply(P, modulo)
    return decipher_enc_complex_matrix(P)

## **Adım 1**
### Aşağıdaki kodlar ile iletişim protokolü başlamadan önce anahtar K seçilmesi ve mod hesaplaması yapılır.

In [10]:
# Shared key K and module value.
K = numpy.array([
    [c(3 + 2j), c(5 + 1j)], 
    [c(7 + 4j), c(3 + 2j)]
])

modulo = table_length()
idm, coefficent = inverse_of_determinant_mod(K, modulo)

print(f"Used modulo for entire algorithm: {modulo}")
print(f"Inverse determinant of complex matrix K in module: {idm}")
print(f"Coefficent of complex matrix K: \n{coefficent}")
print(f"Alice and Bob shared complex key: \n{K}")

Used modulo for entire algorithm: 26
Inverse determinant of complex matrix K in module: 7j
Coefficent of complex matrix K: 
[[ 3.+2.j -5.-1.j]
 [-7.-4.j  3.+2.j]]
Alice and Bob shared complex key: 
[[3.+2.j 5.+1.j]
 [7.+4.j 3.+2.j]]


## **Adım 2**
### Alice ile Bob arasında iletişim protokolü başlar. Gönderici mesajı şifrelemektedir.

In [11]:
message = "Canakkale Onsekiz"
print(f"Message: {message}")
enc_message = encryipt_message(message, modulo, K)

Message: Canakkale Onsekiz
Encrypted Complex Matrix: 
 [[ 3. +1.j 14. +1.j 11.+11.j  1.+12.j]
 [ 5.+15.j 14.+19.j  5.+11.j  9. +0.j]]
Created C matrix as: 
[[17.+11.j 13.+10.j 25.+11.j 24.+21.j]
 [ 2.+22.j 20.+18.j  0. +8.j 12. +2.j]]
Encrypted message as: QKMJYKXUBVTRZHLB


## **Adım 3**
### Alice ile Bob arasında iletişim protokolü devam eder. Alıcı messajı çözer.

In [12]:
dec_message = decrypt_message(K, modulo, enc_message)
print(f"Decrypted message: {dec_message}")

Inverse of K(mod): 
[[12.+21.j  7.+17.j]
 [ 2. +3.j 12.+21.j]]
Decrypted message: CANAKKALEONSEKIZ


### Gönderilen ile çözülen mesaj karşılaştırılır.

In [13]:
message = message.upper()
message = message.replace(" ", "")
print(f"Sended message: {message} \nDecrypted message: {dec_message}")

Sended message: CANAKKALEONSEKIZ 
Decrypted message: CANAKKALEONSEKIZ


### **NOT**
#### Mesajda boşluk gibi beyaz karakterler dışında çift olması gerekiyor. Tek olduğu durumda sona bir karakter ekliyor kod. K matrisi determinantı olan 2x2 karmaşık sayı matrsisi olmalıdır.